#### Tasks

- Try with no embedding layer !!!
- Try dropout, regularization
- Read about GRU, and try it.
- Try loss function as MSE, MAE for neutrals too and with tanh
- Use bias=False for the data that has unbalanced points
- Write a report, what you learned. What is the model you chose. Why you chose it. Steps you've taken
- Test ConvLSTM layer

In [118]:
import glove
import keras
import labeled_functions
import matplotlib.pyplot as plt
import numpy as np
from keras.layers import LSTM, Dense, Input, Embedding, Conv1D, GRU
from keras.layers import MaxPooling1D, GlobalMaxPooling1D, Flatten
from keras.layers.recurrent import LSTM # Don't know the difference
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence, one_hot, Tokenizer
from sklearn.metrics import precision_recall_curve, precision_score, log_loss
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
from sklearn.model_selection import train_test_split

In [2]:
X_corpus, y = labeled_functions.load_with_path(path=r"/Users/ekremguzelyel/Desktop/Assignments/Research/MLLab-IIT/edu/active_learning")

X_sequence = []
for i in X_corpus:
    X_sequence.append(text_to_word_sequence(i))

max_length=0
for i in X_sequence:
    max_length = [len(i) if len(i) > max_length else max_length][0]
print("Max Length:", max_length)

Labeled Data loaded.
Max Length: 123


In [3]:
# One-hot Encoding and Padding Sequences
X_corpus_one_hot = []
for i in X_corpus:
    X_corpus_one_hot.append(one_hot(i, round(max_length*1.2)))
print("One-hot encoded.")

padded_seq_corpus = pad_sequences(X_corpus_one_hot)
print("Sequences padded.")

X_train_padded_seq, X_test_padded_seq, y_train, y_test = train_test_split(padded_seq_corpus, y,
                                                                          test_size=1./3, random_state=42)
print("Train-Test padded sequences split.")

One-hot encoded.
Sequences padded.
Train-Test padded sequences split.


In [8]:
X_train_padded_seq.shape

(2156, 123)

In [18]:
t = Tokenizer()
t.fit_on_texts(X_corpus)
vocab_size = len(t.word_index) + 1
encoded_docs = t.texts_to_sequences(X_corpus)
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='pre')

In [19]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('../../../glove.6B/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [20]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [61]:
# Different combinations tested.
input_layer = Input(shape=(X_train_padded_seq.shape[1],))

e = Embedding(vocab_size,
              100,
              weights=[embedding_matrix],
              input_length=X_train_padded_seq.shape[1],
              trainable=False)(input_layer)
hidden1 = GRU(32, return_sequences=False )(e)

# cnn1 = Conv1D(128, 5, activation='tanh')(hidden1)
# pool1 = MaxPooling1D(5)(cnn1)
# cnn2 = Conv1D(128, 5, activation='sigmoid')(pool1)
# pool2 = GlobalMaxPooling1D()(cnn2)

# hidden2 = Dense(10, activation='relu')(pool2)
output_layer = Dense(1, activation='sigmoid')(hidden1)
model = Model(inputs=input_layer, outputs=output_layer)


model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')
print("Model Compiled")

Model Compiled


In [59]:
X_train_reshape = X_train_padded_seq.reshape(X_train_padded_seq.shape[0], X_train_padded_seq.shape[1],1)

In [115]:
# model.fit(X_test_padded_seq, y_test, epochs=20, batch_size=32)

Epoch 1/20
1079/1079 [==============================] - 3s 3ms/step - loss: 1.5533 - acc: 0.5885
Epoch 2/20
1079/1079 [==============================] - 3s 2ms/step - loss: 0.6896 - acc: 0.6469
Epoch 3/20
1079/1079 [==============================] - 3s 3ms/step - loss: 0.5867 - acc: 0.6932
Epoch 4/20
1079/1079 [==============================] - 3s 3ms/step - loss: 0.5410 - acc: 0.7257
Epoch 5/20
1079/1079 [==============================] - 3s 3ms/step - loss: 0.5068 - acc: 0.7572
Epoch 6/20
1079/1079 [==============================] - 3s 3ms/step - loss: 0.4818 - acc: 0.7683
Epoch 7/20
1079/1079 [==============================] - 3s 3ms/step - loss: 0.4471 - acc: 0.7887
Epoch 8/20
1079/1079 [==============================] - 3s 3ms/step - loss: 0.4324 - acc: 0.7980
Epoch 9/20
1079/1079 [==============================] - 3s 3ms/step - loss: 0.4031 - acc: 0.8221
Epoch 10/20
1079/1079 [==============================] - 3s 3ms/step - loss: 0.3694 - acc: 0.8434
Epoch 11/20
1079/1079 [======

KeyboardInterrupt: 

In [124]:
print(X_train_reshape.shape)
model.fit(X_train_padded_seq, y_train, epochs=20, batch_size=32)

(2156, 123, 1)
Epoch 1/20
2156/2156 [==============================] - 7s 3ms/step - loss: 0.4360 - acc: 0.8043
Epoch 2/20
2156/2156 [==============================] - 6s 3ms/step - loss: 0.2505 - acc: 0.8956
Epoch 3/20
2156/2156 [==============================] - 7s 3ms/step - loss: 0.1209 - acc: 0.9615
Epoch 4/20
2156/2156 [==============================] - 5s 2ms/step - loss: 0.0688 - acc: 0.9828
Epoch 5/20
2156/2156 [==============================] - 7s 3ms/step - loss: 0.0356 - acc: 0.9958
Epoch 6/20
2156/2156 [==============================] - 7s 3ms/step - loss: 0.0249 - acc: 0.9981
Epoch 7/20
2156/2156 [==============================] - 6s 3ms/step - loss: 0.0192 - acc: 0.9995
Epoch 8/20
2156/2156 [==============================] - 6s 3ms/step - loss: 0.0153 - acc: 0.9995
Epoch 9/20
2156/2156 [==============================] - 6s 3ms/step - loss: 0.0130 - acc: 0.9995
Epoch 10/20
2156/2156 [==============================] - 8s 4ms/step - loss: 0.0111 - acc: 0.9995
Epoch 11/20
21

In [125]:
preds = model.predict(X_test_padded_seq) # It gives probabilities

In [126]:
preds_train= (model.predict(X_train_padded_seq))

In [127]:
# Convert predictions to binary.
threshold, pred = 0.25, []
threshold_train, pred_train = 0.25, []
[pred.append(1 if preds[i]>threshold else 0) for i,j in enumerate(preds)]
[pred_train.append(1 if preds_train[i]>threshold else 0) for i,j in enumerate(preds_train)]

print(log_loss(y_test,pred), log_loss(y_train, pred_train))
score_test = model.evaluate(X_test_padded_seq, y_test)
score_train = model.evaluate(X_train_padded_seq, y_train)
score_test, score_train

precision, recall, f1, support = precision_recall_fscore_support(y_test, pred)

print("\nTestAcc", "TrainAcc", "Prec", "Rec", "F1","Sup\n", score_test[1], score_train[1], precision, recall, f1, support, sep="  ")

9.699144007089284 0.016020211499231458
2156/2156 [==============================] - 1s 584us/step

TestAcc  TrainAcc  Prec  Rec  F1  Sup
  0.7219647822609867  1.0  [0.73184358 0.70664207]  [0.71195652 0.72675522]  [0.72176309 0.71655753]  [552 527]


In [84]:
# Different combinations tested.
input_layer = Input(shape=(X_train_padded_seq.shape[1],))

e = Embedding(vocab_size,
              100,
              weights=[embedding_matrix],
              input_length=X_train_padded_seq.shape[1],
              trainable=False)(input_layer)
hidden1 = GRU(32, return_sequences=False )(e)
# cnn1 = Conv1D(128, 5, activation='tanh')(hidden1)
# pool1 = MaxPooling1D(5)(cnn1)
# cnn2 = Conv1D(128, 5, activation='sigmoid')(pool1)
# pool2 = GlobalMaxPooling1D()(cnn2)

hidden2 = Dense(10, activation='tanh')(hidden1)
output_layer = Dense(1, activation='sigmoid')(hidden2)
model2 = Model(inputs=input_layer, outputs=output_layer)


model2.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')
print("Model Compiled")

Model Compiled


In [85]:
model2.fit(X_train_padded_seq, y_train, epochs=20, batch_size=32)

Epoch 1/20
2156/2156 [==============================] - 7s 3ms/step - loss: 0.7005 - acc: 0.5097
Epoch 2/20
2156/2156 [==============================] - 6s 3ms/step - loss: 0.6794 - acc: 0.5640
Epoch 3/20
2156/2156 [==============================] - 7s 3ms/step - loss: 0.6646 - acc: 0.5840
Epoch 4/20
2156/2156 [==============================] - 6s 3ms/step - loss: 0.6484 - acc: 0.6109
Epoch 5/20
2156/2156 [==============================] - 6s 3ms/step - loss: 0.6268 - acc: 0.6489
Epoch 6/20
2156/2156 [==============================] - 6s 3ms/step - loss: 0.6198 - acc: 0.6572
Epoch 7/20
2156/2156 [==============================] - 6s 3ms/step - loss: 0.6010 - acc: 0.6814
Epoch 8/20
2156/2156 [==============================] - 6s 3ms/step - loss: 0.5899 - acc: 0.6934
Epoch 9/20
2156/2156 [==============================] - 6s 3ms/step - loss: 0.5713 - acc: 0.7032
Epoch 10/20
2156/2156 [==============================] - 6s 3ms/step - loss: 0.5584 - acc: 0.7194
Epoch 11/20
2156/2156 [======

In [86]:
preds2 = model2.predict(X_test_padded_seq) # It gives probabilities

In [89]:
# Convert predictions to binary.
threshold2, pred2 = 0.4, []
[pred2.append(1 if preds2[i]>threshold2 else 0) for i,j in enumerate(preds)]

score_test2 = model2.evaluate(X_test_padded_seq, y_test)
score_train2 = model2.evaluate(X_train_padded_seq, y_train)
score_test2, score_train2

precision2, recall2, f12, support2 = precision_recall_fscore_support(y_test, pred2)

print("\nTestAcc", "TrainAcc", "Prec", "Rec", "F1","Sup\n", score_test2[1], score_train2[1], precision2, recall2, f12, support2, sep="  ")

2156/2156 [==============================] - 1s 585us/step

TestAcc  TrainAcc  Prec  Rec  F1  Sup
  0.638554217143673  0.8923933209647495  [0.6619469 0.6536965]  [0.67753623 0.63757116]  [0.66965085 0.64553314]  [552 527]


Different combinations

In [91]:
# Different combinations tested.
input_layer = Input(shape=(X_train_padded_seq.shape[1],))

e = Embedding(vocab_size,
              100,
              weights=[embedding_matrix],
              input_length=X_train_padded_seq.shape[1],
              trainable=False)(input_layer)
hidden1 = GRU(32, return_sequences=True )(e)
hidden2 = GRU(32, return_sequences=False )(hidden1)

# cnn1 = Conv1D(128, 5, activation='tanh')(hidden1)
# pool1 = MaxPooling1D(5)(cnn1)
# cnn2 = Conv1D(128, 5, activation='sigmoid')(pool1)
# pool2 = GlobalMaxPooling1D()(cnn2)

hidden3 = Dense(10, activation='tanh')(hidden2)
output_layer = Dense(1, activation='sigmoid')(hidden3)
model3 = Model(inputs=input_layer, outputs=output_layer)


model3.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')
print("Model Compiled")

Model Compiled


In [92]:
model3.fit(X_train_padded_seq, y_train, epochs=20, batch_size=32)

Epoch 1/20
2156/2156 [==============================] - 13s 6ms/step - loss: 0.6976 - acc: 0.5204
Epoch 2/20
2156/2156 [==============================] - 10s 5ms/step - loss: 0.6701 - acc: 0.5765
Epoch 3/20
2156/2156 [==============================] - 11s 5ms/step - loss: 0.6458 - acc: 0.6132
Epoch 4/20
2156/2156 [==============================] - 10s 5ms/step - loss: 0.6332 - acc: 0.6294
Epoch 5/20
2156/2156 [==============================] - 11s 5ms/step - loss: 0.6115 - acc: 0.6609
Epoch 6/20
2156/2156 [==============================] - 10s 5ms/step - loss: 0.5963 - acc: 0.6712
Epoch 7/20
2156/2156 [==============================] - 11s 5ms/step - loss: 0.5788 - acc: 0.7008
Epoch 8/20
2156/2156 [==============================] - 10s 5ms/step - loss: 0.5685 - acc: 0.7036
Epoch 9/20
2156/2156 [==============================] - 10s 5ms/step - loss: 0.5464 - acc: 0.7208
Epoch 10/20
2156/2156 [==============================] - 11s 5ms/step - loss: 0.5365 - acc: 0.7370
Epoch 11/20
2156/21

In [94]:
preds3 = model3.predict(X_test_padded_seq) # It gives probabilities

In [95]:
# Convert predictions to binary.
threshold3, pred3 = 0.4, []
[pred3.append(1 if preds3[i]>threshold3 else 0) for i,j in enumerate(preds)]

score_test3 = model3.evaluate(X_test_padded_seq, y_test)
score_train3 = model3.evaluate(X_train_padded_seq, y_train)
score_test3, score_train3

precision3, recall3, f13, support3 = precision_recall_fscore_support(y_test, pred3)

print("\nTestAcc", "TrainAcc", "Prec", "Rec", "F1","Sup\n", score_test3[1], score_train3[1], precision3, recall3, f13, support3, sep="  ")

2156/2156 [==============================] - 2s 904us/step

TestAcc  TrainAcc  Prec  Rec  F1  Sup
  0.6116774785949524  0.89053803350576  [0.60944882 0.62837838]  [0.70108696 0.52941176]  [0.65206403 0.57466529]  [552 527]
